# Tutorial for 2-photon calcium imaging and holographic optical stimulation dataset.

This tutorial shows how to access the *Two Photon dataset* using `pynwb`. 

This dataset contains the 2-photon calcium imaging holographic stimulation data and visual stimuli events.

Contents:

- [Reading an NWB file](#read-nwb)
- [Access subject and task metadata](#access-subject)
- [Access Imaging](#access-imaging)
- [Access Holographic Stimulus](#access-holostim)
- [Access Visual Stimulus](#access-visualstim)

A schematic representation where the source data is saved in NWB:

![Alt text](./conversion_outline_diagram.png)

# Reading an NWB file <a name="read-nwb"></a>

This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO
# Choose which epoch to explore
epoch_name = "5stim"
session_id = "w57-1-2023119"
# The file path to a .nwb file
nwbfile_path = f"/media/amtra/Samsung_T5/CN_data/MouseV1-conversion_nwb/test/nwb_stub/{session_id}/{session_id}-{epoch_name}.nwb"
io = NWBHDF5IO(path=nwbfile_path, mode="r", load_namespaces=True)
nwbfile = io.read()

nwbfile

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from the ScanImage metadata (`epoch`) of the first .tiff of the epoch.

In [ ]:
nwbfile.session_start_time

# Access subject metadata <a name="access-subject"></a>

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

# Access TwoPhoton Imaging <a name="access-imaging"></a>

This section demonstraces how to access the raw Two Photon imaging data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw TwoPhoton Imaging

The raw TwoPhoton imaging data is stored in `pynwb.ophys.TwoPhotonSeries` objects (for each channel and plane separately) which is added to `nwbfile.acquisition`. The data can be accessed as `nwbfile.acquisition['TwoPhotonSeriesChannel_number_Plane_number_']`.

The data in [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
names_of_photon_series = nwbfile.acquisition.keys()
_ = [print(name) for name in names_of_photon_series]

In [ ]:
channel_plane_combination = "Channel1Plane0"

In [ ]:
photon_series = nwbfile.acquisition[f"TwoPhotonSeries{channel_plane_combination}"]

In [ ]:
# Visualize the imaging data.

from matplotlib import pyplot as plt

plt.imshow(photon_series.data[50].T, aspect="auto", cmap="RdYlBu_r")
plt.title("TwoPhotonSeries")
plt.show()


In [ ]:
photon_series.rate

In [ ]:
photon_series.starting_time

## Accessing the segmentation data

The segmentation output for the Two Photon Imaging data is stored in `nwbfile.processing["ophys"]`. 

In NWB, the [PlaneSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.PlaneSegmentation) class stores the detected regions of interest in the [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) data. The [ImageSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImageSegmentation) can contain multiple `PlaneSegmentation` tables, so that we can store results of different segmentation algorithms or different segmentation classes.

We can access the plane segmentation for the [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) data as 
`nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentationChannel_number_Plane_number_"]`.


In [ ]:
names_of_plane_segmentation = nwbfile.processing["ophys"]["ImageSegmentation"].plane_segmentations.keys()
_ = [print(name) for name in names_of_plane_segmentation]

In [ ]:
plane_segmentation = nwbfile.processing["ophys"]["ImageSegmentation"][f"PlaneSegmentation{channel_plane_combination}"][:]
plane_segmentation[:10]

In [ ]:
plt.imshow(photon_series.data[50].T, aspect="auto", cmap="RdYlBu_r")
plt.title("TwoPhotonSeries")
plt.show()

plt.imshow(plane_segmentation.image_mask[1].T, aspect="auto", cmap="RdYlBu_r")
plt.title("Image mask (single ROI)")
plt.show()


The summary images of the segmentation are stored in [Images](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.Images) container in NWB. 


In [ ]:
images = nwbfile.processing["ophys"]["SegmentationImages"]
images

In [ ]:
plt.imshow(photon_series.data[50].T, cmap="RdYlBu_r")
plt.title("TwoPhotonSeries")
plt.show()

plt.imshow(images.images[f"CorrelationImage{channel_plane_combination}"].data[:].T, cmap="RdYlBu_r")
plt.title("Image Correlation")
plt.show()

plt.imshow(images.images[f"MeanImage{channel_plane_combination}"].data[:].T, cmap="RdYlBu_r")
plt.title("Image mean")
plt.show()

The fluroscence traces are stored in a [Fluorescence](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.Fluorescence) container, the raw traces can be accessed as `nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"]`.

In [ ]:
nwbfile.processing["ophys"]["Fluorescence"].roi_response_series

## Visualize raw traces

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

data = nwbfile.processing["ophys"]["Fluorescence"][f"RoiResponseSeries{channel_plane_combination}"].data[:, :10]
rate = nwbfile.processing["ophys"]["Fluorescence"][f"RoiResponseSeries{channel_plane_combination}"].rate
df = pd.DataFrame(data)
df["time"] = np.linspace(0, data.shape[0]*rate,data.shape[0])
df.set_index("time", inplace=True)
df.columns.name = 'ROIs'

import plotly.express as px

fig = px.line(df, facet_row="ROIs", facet_row_spacing=0.01)

# hide and lock down axes
fig.update_xaxes(visible=True, fixedrange=False)
fig.update_yaxes(visible=False, fixedrange=False)

# remove facet/subplot labels
fig.update_layout(annotations=[], overwrite=True)

# strip down the rest of the plot
fig.update_layout(
    showlegend=True,
    plot_bgcolor="white",
    margin=dict(t=10, l=10, b=10, r=10)
)

fig.show(config=dict(displayModeBar=True))

In [ ]:
roi_index = 10
if plane_segmentation["Accepted"].values[roi_index]:
    description = "(accepted)"
else:
    description = "(rejected)"

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,5))
data_roi_trace = nwbfile.processing["ophys"]["Fluorescence"][f"RoiResponseSeries{channel_plane_combination}"].data[:, roi_index]
data_neuropil_trace = nwbfile.processing["ophys"]["Fluorescence"][f"Neuropil{channel_plane_combination}"].data[:, roi_index]
rate = nwbfile.processing["ophys"]["Fluorescence"][f"RoiResponseSeries{channel_plane_combination}"].rate
time = np.linspace(0, data.shape[0]*rate,data.shape[0])
plt.plot(time, data_roi_trace, label="activity")
plt.plot(time, data_neuropil_trace, label="neuropil")
plt.xlabel("Time (s)")
plt.ylabel("Df/f")
plt.title(f"ROI {roi_index} {description}")
plt.legend()


# Access Holographic Stimulus <a name="access-holostim"></a>

This section demonstrates how to access the holographic stimulation data and metadata.

The metadata about the stimulus pattern can be accessed as `nwbfile.lab_meta_data["TemporalFocusing"]`.

In [ ]:
nwbfile.lab_meta_data["TemporalFocusing"]

The metadata about the laser and the spatial light modulator can be accessed as `nwbfile.devices["LightSource"]` and `nwbfile.devices["SpatialLightModulator2D"]`. 

In [ ]:
nwbfile.devices["LightSource"]

In [ ]:
nwbfile.devices["SpatialLightModulator2D"]

The metadata about the optogenetic stimulus site can be accessed as `nwbfile.devices["OptogeneticStimulusSite"]

In [ ]:
nwbfile.ogen_sites["OptogeneticStimulusSite"]

The holograms are defined in `nwbfile.lab_meta_data`, and can be accessed by their name: `Hologram_number_`

In [ ]:
for object in nwbfile.lab_meta_data:
    if object.startswith("Hologram"):
        print(object)

In [ ]:
nwbfile.lab_meta_data["Hologram0"].targeted_rois[:]


In [ ]:
nwbfile.lab_meta_data["Hologram0"].segmented_rois[:]


Visualize targeted (in red) and corresponding segmented (in yellow) ROIs on the respective plane (background images are the maximum projection for each plane)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(20, 15))
hologram = "Hologram4"
# Maximum projection here is used as background
import numpy as np
photon_series = nwbfile.acquisition[f"TwoPhotonSeriesChannel1Plane0"]
background_plane0 = np.sum(photon_series.data[:,:,:], axis=0)
photon_series = nwbfile.acquisition[f"TwoPhotonSeriesChannel1Plane1"]
background_plane1 = np.sum(photon_series.data[:,:,:], axis=0)
photon_series = nwbfile.acquisition[f"TwoPhotonSeriesChannel1Plane2"]
background_plane2 = np.sum(photon_series.data[:,:,:], axis=0)

ax0.imshow(background_plane0.T, cmap="grey")
ax0.set_title("Plane0")
ax1.imshow(background_plane1.T, cmap="grey")
ax1.set_title("Plane1")
ax2.imshow(background_plane2.T, cmap="grey")
ax2.set_title("Plane2")
for roi in nwbfile.lab_meta_data[hologram].targeted_rois[:]["voxel_mask"]:
    X = roi[0][0]
    Y = roi[0][1]
    Z = roi[0][2]
    if Z == 0:
        ax0.scatter(X,Y,s=9,c="red",marker='*')
    if Z == 34:
        ax1.scatter(X,Y,s=9,c="red",marker='*')
    if Z == 60:
        ax2.scatter(X,Y,s=9,c="red",marker='*')

for roi in nwbfile.lab_meta_data[hologram].segmented_rois[:]["voxel_mask"]:
    X = roi[0][0]
    Y = roi[0][1]
    Z = roi[0][2]
    if Z == 0:
        ax0.scatter(X,Y,s=13,c="yellow",marker='o', alpha=0.5)
    if Z == 1:
        ax1.scatter(X,Y,s=13,c="yellow",marker='o', alpha=0.5)
    if Z == 2:
        ax2.scatter(X,Y,s=13,c="yellow",marker='o', alpha=0.5)    

fig.show()


The holographic stimulation data is added to `nwbfile.intervals['PatternedOptogeneticStimulusTable']`. 

In [ ]:
stimulus_table = nwbfile.intervals['PatternedOptogeneticStimulusTable']
stimulus_table[:5]

Visualize stimulus on set for a the targeted ROIs in `Hologram0`

In [ ]:
hologram = nwbfile.lab_meta_data["Hologram0"]
stimulus_table_df = stimulus_table.to_dataframe()
fig, ax = plt.subplots()
for i in stimulus_table_df.index[stimulus_table_df["targets"] == hologram][:10]:
    start_time = stimulus_table["start_time"][i]
    stop_time = stimulus_table["stop_time"][i]
    frequency = stimulus_table["frequency"][i]
    power = stimulus_table["power_per_roi"][i]
    for roi, p in zip(np.arange(len(hologram.targeted_rois)), power):
        ax.hlines(y=roi, xmin=start_time, xmax=stop_time, linewidth=p * 100)
    plt.yticks(np.arange(len(hologram.targeted_rois)), list(hologram.targeted_rois[:]["global_ids"]))
    plt.ylabel("Global IDs for targeted rois")
    plt.xlabel("Time")
    plt.grid(visible=True)

# Access Visual Stimulus <a name="access-visualstim"></a>

This section demonstrates how to access the visual stimulus data.

In [ ]:
#need to open a epoch that contains visual stimulus 
epoch_name = "4ori"
# The file path to a .nwb file
nwbfile_path = f"/media/amtra/Samsung_T5/CN_data/MouseV1-conversion_nwb/nwb_stub/{session_id}/{session_id}-{epoch_name}.nwb"
io = NWBHDF5IO(path=nwbfile_path, mode="r", load_namespaces=True)
nwbfile = io.read()

In [ ]:
nwbfile.intervals["VisualStimuli"]